In [1]:
import json
import pickle
from pprint import pprint
import gensim
import re
from gensim.models.doc2vec import LabeledSentence
import numpy as np
import pandas as pd

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [28]:
def create_subset(num_stars):
    dirname = str(num_stars)+'star'
    reviews = []
    i = 0
    filectr = 0
    with open('../yelp_academic_dataset_review.json') as f:
        for line in f:
            review = json.loads(line)
            if review['stars'] == num_stars:
                reviews.append(review)
                i += 1
            if i == 25000:
                with open(dirname+'/reviews-subset-'+str(filectr)+'.pkl','w') as fw:
                    pickle.dump(reviews,fw)
                reviews = []
                filectr += 1
                i = 0
        with open(dirname+'/reviews-subset-'+str(filectr)+'.pkl','w') as fw:
            pickle.dump(reviews,fw) 

In [30]:
create_subset(1)
create_subset(2)
create_subset(3)
create_subset(4)
create_subset(5)

In [35]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

class Reviews(object):
    def __init__(self, filename):
        self.filename = filename
        self.reviews = pickle.load(open(filename,'r'))

    def __iter__(self):
        for uid, review in enumerate(self.reviews):
            #clean_review = clean_str(review['text'])
            yield LabeledSentence(words=review['text'].split(), tags=['REV_%s_%s' % (review['stars'] , uid) ])

In [38]:
reviews = Reviews('1star/reviews-subset-0.pkl')
docmodel_1star = gensim.models.Doc2Vec(reviews, size=100, window=8, min_count=5, workers=4)

In [39]:
reviews1 = Reviews('2star/reviews-subset-0.pkl')
docmodel_2star = gensim.models.Doc2Vec(reviews, size=100, window=8, min_count=5, workers=4)
reviews = Reviews('3star/reviews-subset-0.pkl')
docmodel_3star = gensim.models.Doc2Vec(reviews, size=100, window=8, min_count=5, workers=4)
reviews1 = Reviews('4star/reviews-subset-0.pkl')
docmodel_4star = gensim.models.Doc2Vec(reviews, size=100, window=8, min_count=5, workers=4)
reviews = Reviews('5star/reviews-subset-0.pkl')
docmodel_5star = gensim.models.Doc2Vec(reviews, size=100, window=8, min_count=5, workers=4)

In [55]:
docmodel_1star.save('1star/model-for-file-0.mdl')
docmodel_2star.save('2star/model-for-file-0.mdl')
docmodel_3star.save('3star/model-for-file-0.mdl')
docmodel_4star.save('4star/model-for-file-0.mdl')
docmodel_5star.save('5star/model-for-file-0.mdl')

In [4]:
docmodel_1star = gensim.models.Doc2Vec.load('1star/model-for-file-0.mdl')
docmodel_2star = gensim.models.Doc2Vec.load('2star/model-for-file-0.mdl')
docmodel_3star = gensim.models.Doc2Vec.load('3star/model-for-file-0.mdl')
docmodel_4star = gensim.models.Doc2Vec.load('4star/model-for-file-0.mdl')
docmodel_5star = gensim.models.Doc2Vec.load('5star/model-for-file-0.mdl')

In [11]:
models = [docmodel_1star,docmodel_2star,docmodel_3star,docmodel_4star,docmodel_5star]

In [23]:
len(docmodel_5star.vocab)

21521

In [14]:
test_sentences = pickle.load(open('test-sentences.pkl'))

In [15]:
# sentence splitter
alteos = re.compile(r'([!\?])')
def sentences(l):
    l = alteos.sub(r' \1 .', l).rstrip("(\.)*\n")
    return l.split(".")

docs = [sentences(test_sent) for test_sent in test_sentences]

In [17]:
def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

prob = docprob(docs,models)

In [39]:
docmodel_5star.score('. '.join(docs[0]),1)

array([-11.98872757], dtype=float32)